## 超解像_Skip Connections

### chapter10 と同じデータを使い、モデルをもう一つ複雑なモデルにします

### this model can not work well by now

In [1]:
import os
import glob
import math
import random
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt

from tensorflow.python import keras
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model,Sequential
from tensorflow.python.keras.layers import Conv2D,Dense,Input,MaxPooling2D, UpSampling2D,Lambda
from tensorflow.python.keras.preprocessing.image import load_img,img_to_array,array_to_img,ImageDataGenerator

In [2]:
def drop_resolution(x,scale=3.0):
    size = (x.shape[0],x.shape[1])
    small_size = (int(size[0]/scale), int(size[1]/scale))
    img = array_to_img(x)
    small_img = img.resize(small_size,3)
    return img_to_array(small_img.resize(img.size,3))

In [3]:
from scipy import ndimage
import scipy
def data_generator(data_dir,mode,scale=2.0,target_size=(200,200),batch_size=32,shuffle = True):
    for imgs in ImageDataGenerator().flow_from_directory(
        directory = data_dir,
        classes = [mode],
        class_mode = None,
        color_mode = 'rgb',
        target_size = target_size,
        batch_size = batch_size,
        shuffle = shuffle
    ):
        x = np.array([drop_resolution(img,scale) for img in imgs])
        yield x/255.,imgs/255.

In [4]:
DATA_DIR = 'img_c10'
N_TRAIN_DATA = 1000
N_TEST_DATA = 100
BATCH_SIZE = 20
train_data_generator = data_generator(DATA_DIR, 'train', batch_size = BATCH_SIZE)
test_x,test_y = next(data_generator(
        DATA_DIR,
        'test',
        #target_size = (10,10),
        batch_size = N_TEST_DATA,
        shuffle = False
    ))

Found 100 images belonging to 1 classes.


### Create new model

In [5]:
from tensorflow.python.keras.layers import Add
from tensorflow.python.keras.layers import Conv2DTranspose
inputs = Input((None,None,3), dtype = 'float')
#Encoder
conv1 = Conv2D(64,3,padding= 'same')(inputs)
conv1 = Conv2D(64,3,padding ='same')(conv1)
conv2 = Conv2D(64,3,strides = 2,padding = 'same')(conv1)
conv2 = Conv2D(64,3,padding = 'same')(conv2)
conv3 = Conv2D(64,3,strides = 2, padding = 'same')(conv2)
conv3 = Conv2D(64,3,padding='same')(conv3)

#Decoder
deconv3 = Conv2DTranspose(64,3,padding='same')(conv3)
deconv3 = Conv2DTranspose(64,3,strides = 2, padding= 'same')(deconv3)

#use add() layer to realize the skip connection function
merge2 = Add()([deconv3,conv2])
deconv2 = Conv2DTranspose(64,3,padding='same')(merge2)
deconv2 = Conv2DTranspose(64,3,strides =2,padding='same')(deconv2)
merge1 = Add()([deconv2,conv1])
deconv1 = Conv2DTranspose(64,3,padding='same')(merge1)
deconv1 = Conv2DTranspose(3,3,padding='same')(deconv1)
output = Add()([deconv1,inputs])
model = Model(inputs,output)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 1 448         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 1 2320        conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, None, None, 1 2320        conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (

In [6]:
def psnr(y_true,y_pred):
    return -10*K.log(
        K.mean(K.flatten((y_true - y_pred))**2)
    )/np.log(10)

In [7]:
model.compile(
    loss = 'mean_squared_error',
    optimizer = 'adam',
    metrics = [psnr]
)

In [8]:
config = tf.ConfigProto()
config.gpu_options.allocator_type ='BFC'
config.gpu_options.per_process_gpu_memory_fraction = 0.50

In [ ]:
model.fit_generator(
    train_data_generator,
    validation_data = (test_x,test_y),
    steps_per_epoch = N_TRAIN_DATA//BATCH_SIZE,
    epochs = 50
)

Epoch 1/50
Found 1000 images belonging to 1 classes.


In [ ]:
pred = model.predict(test_x)
from IPython.display import display_png
for i in range(test_x.shape[0]):
    display_png(array_to_img(test_x[i]))
    display_png(array_to_img(test_y[i]))
    display_png(array_to_img(pred[i]))
    print('-'*25)
    if i == 2:
        break